<a href="https://colab.research.google.com/github/Vishal487/ML-Premade-Estimator/blob/master/Premade_Estimators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This tutorial shows you how to solve the Iris classification problem in TensorFlow using Estimators. An Estimator is TensorFlow's high-level representation of a complete model, and it has been designed for easy scaling and asynchronous training.

In [ ]:
import pandas as pd
import tensorflow as tf


## The data set
The sample program in this document builds and tests a model that classifies Iris flowers into three different species based on the size of their sepals and petals.

You will train a model using the Iris data set. The Iris data set contains four features and one label. The four features identify the following botanical characteristics of individual Iris flowers:

* sepal length
* sepal width
* petal length
* petal width

In [ ]:
CSV_COLUMN_NAMES = ['Sepallength', 'Sepalwidth', 'Petallength', 'Petalwidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Viginica']

In [ ]:
train_path = tf.keras.utils.get_file("iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file("iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

In [ ]:
train.head()

,Sepallength,Sepalwidth,Petallength,Petalwidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [ ]:
test.head()

,Sepallength,Sepalwidth,Petallength,Petalwidth,Species
0,5.9,3.0,4.2,1.5,1
1,6.9,3.1,5.4,2.1,2
2,5.1,3.3,1.7,0.5,0
3,6.0,3.4,4.5,1.6,1
4,5.5,2.5,4.0,1.3,1


In [ ]:
# SEPARATING label

train_y = train.pop('Species')
test_y = test.pop('Species')


In [ ]:
train_y.value_counts()

2    42
0    42
1    36
Name: Species, dtype: int64

In [ ]:
# now see all the features

train.head()

,Sepallength,Sepalwidth,Petallength,Petalwidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [ ]:
test.head()

,Sepallength,Sepalwidth,Petallength,Petalwidth
0,5.9,3.0,4.2,1.5
1,6.9,3.1,5.4,2.1
2,5.1,3.3,1.7,0.5
3,6.0,3.4,4.5,1.6
4,5.5,2.5,4.0,1.3


## Estimators

To write a TensorFlow program based on pre-made Estimators, one must perform the following tasks:

* Create one or more input functions.
* Define the model's feature columns.
* Instantiate an Estimator, specifying the feature columns and various hyperparameters.
* Call one or more methods on the Estimator object, passing the appropriate input function as the source of the data.

### Creating input function

An input function is a function that returns a `tf.data.Dataset` object which outputs the following two-element tuple:

* `features` - A Python dictionary in which:
  * Each key is the name of a feature.
  * Each value is an array containing all of that feature's values.
* `label` - An array containing the values of the `label` for every example.

In [ ]:
def input_fn(features, labels=None, training=True, testing=False, batch_size=256):
    """ An input function for training, evaluation and testing """

    # convert the inputs to Dataset.
    if not testing:
        dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    else:
        dataset = tf.data.Dataset.from_tensor_slices((dict(features)))

    # shuffle and repeat if dataset is for training.
    if training:
        dataset = dataset.shuffle(1000).repeat()

    return dataset.batch(batch_size)

### Define feature columns

In [ ]:
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

In [ ]:
train.keys()

Index(['Sepallength', 'Sepalwidth', 'Petallength', 'Petalwidth'], dtype='object')

In [ ]:
my_feature_columns

[NumericColumn(key='Sepallength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Sepalwidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Petallength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Petalwidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

## Instantiate an Estimator

The Iris problem is a classic classification problem. Fortunately, TensorFlow provides several pre-made classifier Estimators, including:

* `tf.estimator.DNNClassifier` for deep models that perform multi-class classification.
* `tf.estimator.DNNLinearCombinedClassifier` for wide & deep models.
* `tf.estimator.LinearClassifier` for classifiers based on linear models.

For the Iris problem, `tf.estimator.DNNClassifier` seems like the best choice.

In [ ]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each

classifier = tf.estimator.DNNClassifier(feature_columns=my_feature_columns,
                                        hidden_units=[30, 10],
                                        n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpalv8ohpe', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Train, Evaluate, and Predict

### Train the model

In [ ]:
classifier.train(input_fn=lambda: input_fn(train, train_y, training=True),
                 steps=5000) 

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO

### Evaluate the trained model

In [ ]:
eval_result = classifier.evaluate(input_fn= lambda: input_fn(test, test_y, training=False))

print('Evaluation results: ')
print(eval_result)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-04-12T05:36:42Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpalv8ohpe/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.20944s
INFO:tensorflow:Finished evaluation at 2020-04-12-05:36:42
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.93333334, averag

In [ ]:
print('Evaluation results: ')
print(eval_result)

Evaluation results: 
{'accuracy': 0.93333334, 'average_loss': 0.46515837, 'loss': 0.46515837, 'global_step': 5000}


### Making Prediction from the trained model
**on the test dataset**

In [ ]:
predictions = classifier.predict(input_fn= lambda: input_fn(test, training=False, testing=True))

In [ ]:
print(predictions)

<generator object Estimator.predict at 0x7fd5f8d16eb8>


In [ ]:
for pred_dict, expec in zip(predictions, test_y):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%), expected "{}"'. format(SPECIES[class_id], 100*probability, SPECIES[expec]))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpalv8ohpe/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "Versicolor" (51.8%), expected "Versicolor"
Prediction is "Viginica" (65.8%), expected "Viginica"
Prediction is "Setosa" (71.1%), expected "Setosa"
Prediction is "Versicolor" (49.9%), expected "Versicolor"
Prediction is "Versi